In [ ]:
# -----------------------------------------------------------------------------
# Portions of this code are adapted from:
#   - https://github.com/TamaraGrossmann/FEM-vs-PINNs.git
#   - Grossmann, T. G., Komorowska, U. J., Latz, J., & Schönlieb, C.-B. (2023).
#     Can Physics-Informed Neural Networks beat the Finite Element Method?
#     arXiv:2302.04107.
# -----------------------------------------------------------------------------

In [ ]:
# (for Google Colab)
!pip install pyDOE

In [ ]:
import jax, flax, optax, time, pickle
import os
import jax.numpy as np
import numpy as onp
from functools import partial
from pyDOE import lhs
from typing import Sequence
import json
from tensorflow_probability.substrates import jax as tfp

In [ ]:
# Run on the first GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from jax.extend.backend import get_backend
print(get_backend().platform)

# Hyperparameters

In [ ]:
architecture_list = [[20, 20, 20, 1], [100, 100, 100, 1], [500, 500, 500, 1],
                     [20, 20, 20, 20, 1], [100, 100, 100, 100, 1], [500, 500, 500, 500, 1],
                     [20, 20, 20, 20, 20, 1], [100, 100, 100, 100, 100, 1], [500, 500, 500, 500, 500, 1],
                     [20, 20, 20, 20, 20, 20, 1], [100, 100, 100, 100, 100, 100, 1],
                     [500, 500, 500, 500, 500, 500, 1], [20, 20, 20, 20, 20, 20, 20, 1],
                     [100, 100, 100, 100, 100, 100, 100, 1],] # NN architecture list
lr = 1e-4 # learning rate
num_epochs = 50000 # number of training epochs
eps = 0.01

# NN Architecture

In [ ]:
# Define NN architecture
class PDESolution(flax.linen.Module): # inherit from Module class
    features: Sequence[int] # dataclass (e.g. [10, 20, 1])

    @flax.linen.compact # a decorator to define the model in more concise and readable way
    def __call__(self, x): # __call__: makes an object callable, which enables you to use instances of the class like functions
        for feature in self.features[:-1]:
            x = flax.linen.tanh(flax.linen.Dense(feature)(x)) # initialize weights(parameters) w/ default initializer
        # Final Dense layer
        x = flax.linen.Dense(self.features[-1], kernel_init = flax.linen.initializers.glorot_uniform())(x) # initialize weights(parameters) w/ Glorot uniform for stable training
        return x

# Loss Function

In [ ]:
# Hessian-vector product
# (it is more general approach than gradient, even if it doesn't make a change in this 1D problem)
def hvp(f, primals, tangents):
    return jax.jvp(jax.grad(lambda x: f(x)[0]), primals, tangents)[1]

# PDE residual
@partial(jax.vmap, in_axes = (None, 0, 0, None), out_axes = 0)
@partial(jax.jit, static_argnums = (0,))
def residual(u, t, x, eps):
    u_t = jax.jvp(u, (t, x), (1., 0.))[1] # partial derivative w.r.t t = directional derivative along <1, 0>
    u_xx = jax.hessian(u, argnums = 1)(t, x) # differentiate w.r.t argument 1(x)
    return u_t - eps*u_xx + (2/eps)*u(t,x)*(1-u(t,x))*(1-2*u(t,x))

# Inital condition
@partial(jax.vmap, in_axes=0) # vectorized over "xs"
def u_init(xs):
    return np.array([0.25*(np.sin(2*np.pi*xs) + 0.25*np.sin(16*np.pi*xs)) + 0.5])

# Loss functionals
@jax.jit
def pde_residual(params, points):
    return np.mean(residual(lambda t, x: model.apply(params, np.stack((x))), points[:, 0], points[:, 1], eps) ** 2) # Mean Squared Error

@jax.jit(jax.jit, static_argnums=0)
def init_residual(u_init, params, xs):
    lhs = model.apply(params, np.stack((np.zeros_like(xs[:, 0]), xs[:, 0]), axis=1))
    rhs = u_init(xs[:, 0])
    return np.mean((lhs - rhs) ** 2)

@jax.jit
def boundary_residual(params, ts): # u(t, 0) = u(t, 1)
    return np.mean((model.apply(params, np.stack((ts[:, 0], np.zeros_like(ts[:, 0])), axis = 1)) -
                    model.apply(params, np.stack((ts[:, 0], np.ones_like(ts[:, 0])), axis = 1)))**2) # ts : (n, 1)

# Training Loop

In [ ]:
# Define Training Step
@partial(jax.jit, static_argnums = (1,))
def training_step_ini(params, opt, opt_state, key):
    """
    In the PDE problem with initial condition, training NN with I.C. first can be helpful because
    - it does not need so many training steps(epochs). (initial guess is already close to the solution)
    - It stabilizes subsequent training, because the solution is already near the correct initial profile.
    
    Args:
        params: model parameters
        opt: optimizer
        opt_state: optimizer state
        key: random key for sampling
    """
    lb = onp.array([0., 0.]) # lower bound
    ub = onp.array([0.05, 1.]) # upper bound
    # scale the samples from [0, 1] to [lb, ub]
    domain_points = lb + (ub - lb) * lhs(2, 20000) # latin hypercube sampling 20000 points within (ti, xi) ∈ [0, 0.05] × [0, 1]
    boundary_points = lb[0] + (ub[0] - lb[0]) * lhs(1, 250) # latin hypercube sampling 250 points within ti ∈ [0, 0.05]
    init_points = lb[1] + (ub[1] - lb[1]) * lhs(1, 500) # latin hypercube sampling 500 points within xi ∈ [0, 1]

    loss_val, grad = jax.value_and_grad(lambda params: init_residual(u_init,params, init_points))(params)
    update, opt_state = opt.update(grad, opt_state, params) # update using "grad"
    params = optax.apply_updates(params, update) # apply updates to "params"
    return params, opt_state, key, loss_val

@partial(jax.jit, static_argnums = (1,))
def training_step(params, opt, opt_state, key):
    # (same as above)
    lb = onp.array([0., 0.])
    ub = onp.array([0.05, 1.])
    domain_points = lb + (ub - lb) * lhs(2, 20000)
    boundary_points = lb[0] + (ub[0] - lb[0]) * lhs(1, 250)
    init_points = lb[1] + (ub[1] - lb[1]) * lhs(1, 500)

    # Weight factor 1000 for I.C. loss term heuristically gives the best result
    # (total loss will be more sensitive to I.C. loss than other loss terms)
    loss_val, grad = jax.value_and_grad(lambda params: pde_residual(params, domain_points) + 
                                        1000 * init_residual(u_init,params, init_points) +
                                        boundary_residual(params, boundary_points))(params)
    update, opt_state = opt.update(grad, opt_state, params)
    params = optax.apply_updates(params, update)
    return params, opt_state, key, loss_val

# Training loop
def train_loop(params, adam, opt_state, key):
    losses = []
    for i in range(7000):
        params, opt_state, key, loss_val = training_step_ini(params, adam, opt_state, key)
        losses.append(loss_val.item())
    
    for _ in range(num_epochs): # "_" is used because the variable is not used in for loop
        params, opt_state, key, loss_val = training_step(params, adam, opt_state, key)
        losses.append(loss_val.item())
    return losses, params, opt_state, key, loss_val # return final values

# Helper Functions for L-BFGS Wrapper

In [ ]:
# L-BFGS requires the parameters to be a single flattened array!
def concat_params(params): # flatten the parameters
    params, tree = jax.tree_util.tree_flatten(params) # "params" is flattened to a list of arrays
    # "tree" describes the original structure of parameters. It allows to reconstruct the original nested format later.
    shapes = [param.shape for param in params] # shape of each array in the "params" list
    return np.concatenate([param.reshape(-1) for param in params]), tree, shapes # concat to single 1D array

def unconcat_params(params, tree, shapes): # unflatten the parameters
    split_vec = np.split(params, np.cumsum([np.prod(shape) for shape in shapes])) # "np.cumsum" figures out the boundaries where to split the flattened "params"
    split_vec = [vec.reshape(*shape) for vec, shape in zip(split_vec, shapes)] # reshape slices of vector ("*" unpack the tuple into individual arguments)
    return jax.tree_util.tree_unflatten(tree, split_vec)

# Load Ground Truth Solutions

# Train PINN & Approximate Solution